In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
model_name, folder_name = "train_10/","train_10/"
data_directory  = 'drive/MyDrive/Colab Notebooks/Polyamanita/data/'+folder_name
model_directory = 'drive/MyDrive/Colab Notebooks/Polyamanita/models/'+model_name+'savedmodel.h5'

In [ ]:
size = 256
batch_size = 79

In [ ]:
!pip -q install tensorflow tensorflow-gpu opencv-python matplotlib keras

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for tensorflow-gpu ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> tensorflow-gpu

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras import layers
import matplotlib
import pathlib
import cv2
import imghdr
import time
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential, load_model # MAY NEED TO BE REPLACED W/ FUNCTIONAL API
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D
from keras.metrics import Precision, Recall, BinaryAccuracy
from PIL import Image

In [ ]:
model = tf.keras.models.load_model(model_directory)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_directory,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(size, size),
    batch_size=batch_size
)

class_names = train_ds.class_names

Found 3610 files belonging to 10 classes.
Using 2888 files for training.


In [ ]:
print(class_names)

['11082_Xerocomellus_chrysenteron', '12919_Cylindrobasidium_laeve', '14064_Fomitopsis_pinicola', '14160_Ganoderma_pfeifferi', '17233_Mycena_galericulata', '20983_Trametes_versicolor', '21143_Tricholoma_scalpturatum', '40392_Armillaria_lutea', '40985_Byssomerulius_corium', '61207_Coprinellus_micaceus']


In [ ]:
img_url = [
    
    ["40392_Armillaria_lutea","http://www.freenatureimages.eu/plants/Funghi,%20Paddestoelen,%20Mushrooms-Toadstools/Armillaria%20lutea/Armillaria%20lutea%201%2C%20Saxifraga-Jan%20de%20Laat.jpg"],
    ["40392_Armillaria_lutea","https://www.asergeev.com/pictures/archives/2013/1261/jpeg/14b.jpg"],
    ["40392_Armillaria_lutea","https://www.asergeev.com/pictures/archives/2016/1941/jpeg/08b.jpg"],

    ["40985_Byssomerulius_corium","https://btweenblinks.files.wordpress.com/2013/12/p1030106.jpg"],
    ["40985_Byssomerulius_corium","https://api.ispotnature.org/storage/app/sites/default/files/images/38683/31d25304cecb8a8de735fbe22854d0f2.jpg"],
    ["40985_Byssomerulius_corium","https://btweenblinks.files.wordpress.com/2013/12/p1030085.jpg"],

    ["61207_Coprinellus_micaceus","https://www.messiah.edu/Oakes/fungi_on_wood/gilled%20fungi/images/Cop%20micaceus%20DW.jpg"],
    ["61207_Coprinellus_micaceus","https://upload.wikimedia.org/wikipedia/commons/c/c3/Coprinellus_micaceus_Glimmer-Tintling.jpg"],
    ["61207_Coprinellus_micaceus","https://live.staticflickr.com/2947/33650872392_99a1e74718_b.jpg"],
    
    ["12919_Cylindrobasidium_laeve","https://live.staticflickr.com/65535/49484537306_b3f48b4039_b.jpg"],
    ["12919_Cylindrobasidium_laeve","https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2i5lvJBXSqveM-vaaE69DY2AhZId9kyAosQ&usqp=CAU"],
    ["12919_Cylindrobasidium_laeve","https://live.staticflickr.com/7851/45859587244_bd3d8fae5e_b.jpg"],

    ["14064_Fomitopsis_pinicola","https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Fomitopsis_pinicola_1.JPG/1200px-Fomitopsis_pinicola_1.JPG"],
    ["14064_Fomitopsis_pinicola","https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Fomitopsis_pinicola_109142.jpg/1200px-Fomitopsis_pinicola_109142.jpg"],
    ["14064_Fomitopsis_pinicola","https://www.mykoweb.com/CAF/photos/large/Fomitopsis_pinicola%28mgw-02%29.jpg"],

    ["14160_Ganoderma_pfeifferi", "https://www.first-nature.com/fungi/images/ganodermataceae/ganoderma-pfeifferi2.jpg"],
    ["14160_Ganoderma_pfeifferi", "https://www.first-nature.com/fungi/images/ganodermataceae/ganoderma-pfeifferi1.jpg"],
    ["14160_Ganoderma_pfeifferi", "https://c8.alamy.com/comp/DAYX2E/beeswax-bracket-fungus-ganoderma-pfeifferi-ganodermataceae-syn-ganoderma-DAYX2E.jpg"],

    ["17233_Mycena_galericulata","https://www.first-nature.com/fungi/images/mycenaceae/mycena-galericulata1.jpg"],
    ["17233_Mycena_galericulata","https://www.mushroomexpert.com/images/kuo7/mycena_galericulata_03.jpg"],
    ["17233_Mycena_galericulata","https://www.first-nature.com/fungi/images/mycenaceae/mycena-galericulata1.jpg"],

    ["20983_Trametes_versicolor","https://upload.wikimedia.org/wikipedia/commons/a/a3/Trametes_versicolor_G4_%281%29.JPG"],
    ["20983_Trametes_versicolor","https://ars.els-cdn.com/content/image/1-s2.0-S1877117319300353-f15-01-9780128177402.jpg"],
    ["20983_Trametes_versicolor","https://www.messiah.edu/Oakes/fungi_on_wood/poroid%20fungi/images/Tra%20versicolor%20Nathan%20Wilson.jpg"],

    ["21143_Tricholoma_scalpturatum","https://upload.wikimedia.org/wikipedia/commons/7/78/Tricholoma_scalpuratum_20061014wa.jpg"],
    ["21143_Tricholoma_scalpturatum","https://c8.alamy.com/zooms/9/1ac2cda5571c480ca5441eab637130cd/2dk86n8.jpg"],
    ["21143_Tricholoma_scalpturatum","https://www.hlasek.com/foto/tricholoma_scalpturatum_be4903.jpg"],

    ["11082_Xerocomellus_chrysenteron","https://www.mycodb.fr/photos/Xerocomellus_chrysenteron_2014_rp_1.jpg"],
    ["11082_Xerocomellus_chrysenteron","https://www.mykoweb.com/CAF/photos/large/Xerocomellus_diffractus_mgw-02.jpg"],
    ["11082_Xerocomellus_chrysenteron","https://www.mushroomexpert.com/images/kuo5/xerocomellus_chrysenteron_02.jpg"],

]

In [ ]:
for i in img_url:
    try: 
        img_path = tf.keras.utils.get_file(origin=i[1])
        img = tf.keras.preprocessing.image.load_img(
            img_path, target_size=(size, size, 3)
        )
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create a batch
        predictions = model.predict(img_array)
        predictions_sigmoid = tf.nn.sigmoid(predictions)
        score = tf.nn.softmax(predictions[0])
        guessed_name = class_names[np.argmax(score)]
        percent = 100 * np.max(score)
        real_name = i[0]
        print(
            f"Guessed: {guessed_name}\n%Confidence: {percent}.\nReal: {real_name}\n{guessed_name == real_name}\n"
        )
    except Exception as e:
        print(f'Bad: {e}\n{i[1]}')
        continue

1/1 [==============================] - 0s 111ms/step
Guessed: 40392_Armillaria_lutea
%Confidence: 99.07903671264648.
Real: 40392_Armillaria_lutea
True

1/1 [==============================] - 0s 136ms/step
Guessed: 61207_Coprinellus_micaceus
%Confidence: 92.09005236625671.
Real: 40392_Armillaria_lutea
False

1/1 [==============================] - 0s 113ms/step
Guessed: 40392_Armillaria_lutea
%Confidence: 93.11773777008057.
Real: 40392_Armillaria_lutea
True

1/1 [==============================] - 0s 123ms/step
Guessed: 40985_Byssomerulius_corium
%Confidence: 77.26105451583862.
Real: 40985_Byssomerulius_corium
True

Bad: URL fetch failure on https://api.ispotnature.org/storage/app/sites/default/files/images/38683/31d25304cecb8a8de735fbe22854d0f2.jpg: 403 -- Forbidden
https://api.ispotnature.org/storage/app/sites/default/files/images/38683/31d25304cecb8a8de735fbe22854d0f2.jpg
1/1 [==============================] - 0s 122ms/step
Guessed: 40985_Byssomerulius_corium
%Confidence: 59.8352074623